In [1]:
import pandas as pd
import json
from datetime import datetime
import pytz
import os

In [2]:
ts = pd.read_csv('TS.csv')

In [3]:
ts.head()

,Participant,Test Number,Start timestamp,End timestamp,Hand,Back,Msafety_acc,Msafety_ppg,Bangle
0,DS,1,1.698412e+12,1.698412e+12,testResults_Ds01_hand.json,testResults_Ds01.json,6ba8ac15de_st1dby_2023-10-27_13_2023-10-27_17_...,6ba8ac15de_st1dby_2023-10-27_13_2023-10-27_17_...,DS01.json
1,DS,2,1.698412e+12,1.698412e+12,testResults_Ds02_hand.json,testResults_Ds02.json,6ba8ac15de_st1dby_2023-10-27_13_2023-10-27_17_...,6ba8ac15de_st1dby_2023-10-27_13_2023-10-27_17_...,DS02.json
2,DS,3,1.698412e+12,1.698412e+12,testResults_Ds03_hand.json,testResults_Ds03.json,6ba8ac15de_st1dby_2023-10-27_13_2023-10-27_17_...,6ba8ac15de_st1dby_2023-10-27_13_2023-10-27_17_...,DS03.json
3,DS,4,1.698412e+12,1.698412e+12,testResults_Ds04_hand.json,testResults_Ds04.json,6ba8ac15de_st1dby_2023-10-27_13_2023-10-27_17_...,6ba8ac15de_st1dby_2023-10-27_13_2023-10-27_17_...,DS04.json
4,DS,5,1.698413e+12,1.698413e+12,testResults_Ds05_hand.json,testResults_Ds05.json,6ba8ac15de_st1dby_2023-10-27_13_2023-10-27_17_...,6ba8ac15de_st1dby_2023-10-27_13_2023-10-27_17_...,DS05.json


# MSAFETY

In [4]:
participant = ['DS', 'DL', 'MB', 'RC', 'PB', 'LC']

for p in participant :
    time = ts[ts['Participant']==p]
    time.reset_index(drop=True, inplace=True)
    
    msafety_acc = 'Data/'+p+'/msafety/'+time['Msafety_acc'][0]
    acc = pd.read_csv(msafety_acc)
    
    msafety_ppg = 'Data/'+p+'/msafety/'+time['Msafety_ppg'][0]
    ppg = pd.read_csv(msafety_ppg)
    
    for i in range(1,11):
        start = time['Start timestamp'][i-1] - 5000
        end = time['End timestamp'][i-1] + 5000
        
        acc_exp = acc[acc['ts(msec)']>=start]
        acc_exp = acc_exp[acc_exp['ts(msec)']<=end]
        
        acc_exp = acc_exp.rename(columns={'ts(msec)': 'timestamp'})
        acc_exp = acc_exp.rename(columns={'X': 'accGx'})
        acc_exp = acc_exp.rename(columns={'Y': 'accGy'})
        acc_exp = acc_exp.rename(columns={'Z': 'accGz'})
        acc_exp['timestamp'] = acc_exp['timestamp']/1000
        
        ppg_exp = ppg[ppg['ts(msec)']>=start]
        ppg_exp = ppg_exp[ppg_exp['ts(msec)']<=end]
        
        ppg_exp = ppg_exp.rename(columns={'ts(msec)': 'timestamp'})
        ppg_exp = ppg_exp.rename(columns={'PPG': 'ppg'})
        ppg_exp['timestamp'] = ppg_exp['timestamp']/1000
        
        path_acc = 'Results/'+p+'/'+str(i)+'/'+p+'_msafety_acc_'+str(i)+'.csv'
        acc_exp.to_csv(path_acc, index=False)
        
        path_ppg = 'Results/'+p+'/'+str(i)+'/'+p+'_msafety_ppg_'+str(i)+'.csv'
        ppg_exp.to_csv(path_ppg, index=False)
        

# BANGLE

In [7]:
participant = ['DS', 'DL', 'MB', 'RC', 'PB', 'LC']

for p in participant :
    time = ts[ts['Participant']==p]
    time.reset_index(drop=True, inplace=True)
    
    for i in range(1,11):
        start = time['Start timestamp'][i-1] - 5000
        end = time['End timestamp'][i-1] + 5000
        
        # Chemin vers le fichier JSON
        json_file_path_bangle = 'Data/'+p+'/bangle/'+time['Bangle'][i-1]
        
        # Charger les données JSON
        with open(json_file_path_bangle, 'r') as json_file:
            data = json.load(json_file)

        # Créer un DataFrame pour chaque catégorie
        df_accel = pd.DataFrame(data.get('accel', []))
        df_compass = pd.DataFrame(data.get('compass', []))
        df_pressure = pd.DataFrame(data.get('pressure', []))
        df_steps = pd.DataFrame(data.get('steps', []))
        df_hr = pd.DataFrame(data.get('hr',[]))
        
        # Transformation des colonne ms pour avoir des timestamp
        date_string = data.get('startTs')
        utc_time = datetime.fromisoformat(date_string.replace('Z', '+00:00'))
        local_timezone = pytz.timezone('Europe/Paris')  
        local_time = utc_time.astimezone(local_timezone)
        timestamp = local_time.timestamp() * 1000 

        def addTS(x):
            return x + timestamp

        if 'ms' in df_accel.columns :
            df_accel['ms'] = df_accel['ms'].apply(addTS)
        if 'ms' in df_compass.columns :
            df_compass['ms'] = df_compass['ms'].apply(addTS)
        if 'ms' in df_pressure.columns :
            df_pressure['ms'] = df_pressure['ms'].apply(addTS)
        if 'ms' in df_steps.columns :
            df_steps['ms'] = df_steps['ms'].apply(addTS)
        if 'ms' in df_hr.columns :
            df_hr['ms'] = df_hr['ms'].apply(addTS)

        # Couper les DF avec start et end
        if 'ms' in df_accel.columns :
            df_accel = df_accel[df_accel['ms']>=start]
            df_accel = df_accel[df_accel['ms']<=end]
            df_accel = df_accel.rename(columns={'ms': 'timestamp'})
            df_accel = df_accel.rename(columns={'x': 'accGx'})
            df_accel = df_accel.rename(columns={'y': 'accGy'})
            df_accel = df_accel.rename(columns={'z': 'accGz'})
            df_accel['timestamp'] = df_accel['timestamp']/1000
        if 'ms' in df_compass.columns :
            df_compass = df_compass[df_compass['ms']>=start]
            df_compass = df_compass[df_compass['ms']<=end]
            df_compass = df_compass.rename(columns={'ms': 'timestamp'})
            df_compass = df_compass.rename(columns={'x': 'magnRawx'})
            df_compass = df_compass.rename(columns={'y': 'magnRawy'})
            df_compass = df_compass.rename(columns={'z': 'magnRawz'})
            df_compass['timestamp'] = df_compass['timestamp']/1000
        if 'ms' in df_steps.columns :
            df_steps = df_steps[df_steps['ms']>=start]
            df_steps = df_steps[df_steps['ms']<=end]
            df_steps = df_steps.rename(columns={'ms': 'timestamp'})
            df_steps['timestamp'] = df_steps['timestamp']/1000
        if 'ms' in df_hr.columns :
            df_hr = df_hr[df_hr['ms']>=start]
            df_hr = df_hr[df_hr['ms']<=end]
            df_hr = df_hr.rename(columns={'ms': 'timestamp'})
            df_hr['timestamp'] = df_hr['timestamp']/1000

        # Écrire chaque DataFrame dans un csv séparé
        path_accel = 'Results/'+p+'/'+str(i)+'/'+p+'_bangle_accel_'+str(i)+'.csv'
        df_accel.to_csv(path_accel, index=False)
        
        path_compass = 'Results/'+p+'/'+str(i)+'/'+p+'_bangle_compass_'+str(i)+'.csv'
        df_compass.to_csv(path_compass, index=False)
        
        path_pressure = 'Results/'+p+'/'+str(i)+'/'+p+'_bangle_pressure_'+str(i)+'.csv'
        df_pressure.to_csv(path_pressure, index=False)
        
        path_steps = 'Results/'+p+'/'+str(i)+'/'+p+'_bangle_steps_'+str(i)+'.csv'
        df_steps.to_csv(path_steps, index=False)
        
        path_hr = 'Results/'+p+'/'+str(i)+'/'+p+'_bangle_hr_'+str(i)+'.csv'
        df_hr.to_csv(path_hr, index=False)

# BACK

In [3]:
participant = ['DS', 'DL', 'MB', 'RC', 'PB', 'LC']

for p in participant:
    time = ts[ts['Participant'] == p]
    time.reset_index(drop=True, inplace=True)

    for i in range(1, 11):
        start = time['Start timestamp'][i - 1] - 5000
        end = time['End timestamp'][i - 1] + 5000

        # Chemin vers le fichier JSON
        json_file_path_back = 'Data/' + p + '/back/' + time['Back'][i - 1]
        
        # Charger les données JSON
        if time['Back'][i - 1] != 'None':
            with open(json_file_path_back, 'r') as json_file:
                data = json.load(json_file)

            # Créer un DataFrame pour chaque catégorie
            df_motion = pd.DataFrame(data.get('motion', []))
            df_orientation = pd.DataFrame(data.get('orientation', []))
            df_heartRate = pd.DataFrame(data.get('heartRate', []))
            df_cadence = pd.DataFrame(data.get('cadence', []))

            # Transformation des colonnes ms pour avoir des timestamp
            date_string = data.get('startTs')
            utc_time = datetime.fromisoformat(date_string.replace('Z', '+00:00'))
            local_timezone = pytz.timezone('Europe/Paris')
            local_time = utc_time.astimezone(local_timezone)
            timestamp = local_time.timestamp() * 1000

            def addTS(x):
                return x + timestamp

            if 'msFromStart' in df_motion.columns:
                df_motion['msFromStart'] = df_motion['msFromStart'].apply(addTS)
            if 'msFromStart' in df_orientation.columns:
                df_orientation['msFromStart'] = df_orientation['msFromStart'].apply(addTS)
            if 'msFromStart' in df_heartRate.columns:
                df_heartRate['msFromStart'] = df_heartRate['msFromStart'].apply(addTS)
            if 'msFromStart' in df_cadence.columns:
                df_cadence['msFromStart'] = df_cadence['msFromStart'].apply(addTS)

            # Couper les DF avec start et end
            if 'msFromStart' in df_motion.columns:
                df_motion = df_motion[df_motion['msFromStart'] >= start]
                df_motion = df_motion[df_motion['msFromStart'] <= end]
            if 'msFromStart' in df_orientation.columns:
                df_orientation = df_orientation[df_orientation['msFromStart'] >= start]
                df_orientation = df_orientation[df_orientation['msFromStart'] <= end]
            if 'msFromStart' in df_heartRate.columns:
                df_heartRate = df_heartRate[df_heartRate['msFromStart'] >= start]
                df_heartRate = df_heartRate[df_heartRate['msFromStart'] <= end]
            if 'msFromStart' in df_cadence.columns:
                df_cadence = df_cadence[df_cadence['msFromStart'] >= start]
                df_cadence = df_cadence[df_cadence['msFromStart'] <= end]

            # Séparer la colonne 'acc' en 'accx', 'accy', 'accz'
            if 'acc' in df_motion.columns:
                df_motion['accx'] = df_motion['acc'].apply(lambda x: x['x'])
                df_motion['accy'] = df_motion['acc'].apply(lambda x: x['y'])
                df_motion['accz'] = df_motion['acc'].apply(lambda x: x['z'])
                df_motion.drop(columns=['acc'], inplace=True)
                
             # Séparer la colonne 'accG' en 'accGx', 'accGy', 'accGz'
            if 'accG' in df_motion.columns:
                df_motion['accGx'] = df_motion['accG'].apply(lambda x: x['x'])
                df_motion['accGy'] = df_motion['accG'].apply(lambda x: x['y'])
                df_motion['accGz'] = df_motion['accG'].apply(lambda x: x['z'])
                df_motion.drop(columns=['accG'], inplace=True)
                
            # Séparer la colonne 'rotRate' en 'alpha', 'beta', 'gamma'
            if 'rotRate' in df_motion.columns:
                df_motion['alpha'] = df_motion['rotRate'].apply(lambda x: x['alpha'])
                df_motion['beta'] = df_motion['rotRate'].apply(lambda x: x['beta'])
                df_motion['gamma'] = df_motion['rotRate'].apply(lambda x: x['gamma'])
                df_motion.drop(columns=['rotRate'], inplace=True)
                
            # Modifications des noms de colonnes
            if 'msFromStart' in df_motion.columns :
                df_motion = df_motion.rename(columns={'msFromStart': 'timestamp'})
                df_motion['timestamp'] = df_motion['timestamp']/1000
            
            if 'msFromStart' in df_orientation.columns :
                df_orientation = df_orientation.rename(columns={'msFromStart': 'timestamp'})
                df_orientation['timestamp'] = df_orientation['timestamp']/1000
            
            if 'msFromStart' in df_cadence.columns :
                df_cadence = df_cadence.rename(columns={'msFromStart': 'timestamp'})
                df_cadence['timestamp'] = df_cadence['timestamp']/1000
                df_cadence = df_cadence[['timestamp', 'isRunning', 'instantaneousSpeed', 'instantaneousCadence', 'instantaneousStrideLength', 'totalDistance']]
            
            # Écrire chaque DataFrame dans un fichier csv séparé
            directory = 'Results/' + p + '/' + str(i) + '/'
            if not os.path.exists(directory):
                os.makedirs(directory)

            df_motion.to_csv(directory + p + '_back_motion_' + str(i) + '.csv', index=False)
            df_orientation.to_csv(directory + p + '_back_orientation_' + str(i) + '.csv', index=False)
            df_heartRate.to_csv(directory + p + '_back_heartRate_' + str(i) + '.csv', index=False)
            df_cadence.to_csv(directory + p + '_back_cadence_' + str(i) + '.csv', index=False)


# HAND

In [13]:
participant = ['DS', 'DL', 'MB', 'RC', 'PB', 'LC']

for p in participant:
    time = ts[ts['Participant'] == p]
    time.reset_index(drop=True, inplace=True)
    
    for i in range(1, 11):
        start = time['Start timestamp'][i-1] 
        end = time['End timestamp'][i-1] 
        
        # Chemin vers le fichier JSON Hand
        json_file_path_hand = f'Data/{p}/hand/{time["Hand"][i-1]}'
        
        # Vérifier si le fichier JSON Hand existe
        if os.path.exists(json_file_path_hand):
            # Charger les données JSON Hand
            with open(json_file_path_hand, 'r') as json_file:
                data = json.load(json_file)

            # Créer un DataFrame pour chaque catégorie
            df_motion = pd.DataFrame(data.get('motion', []))
            df_orientation = pd.DataFrame(data.get('orientation', []))
            df_heart_rate = pd.DataFrame(data.get('heartRate', []))
            df_cadence = pd.DataFrame(data.get('cadence', []))
            
            # Séparer la colonne 'acc' en 'accx', 'accy', 'accz'
            if 'acc' in df_motion.columns:
                df_motion['accx'] = df_motion['acc'].apply(lambda x: x['x'])
                df_motion['accy'] = df_motion['acc'].apply(lambda x: x['y'])
                df_motion['accz'] = df_motion['acc'].apply(lambda x: x['z'])
                df_motion.drop(columns=['acc'], inplace=True)
                
             # Séparer la colonne 'accG' en 'accx', 'accy', 'accz'
            if 'accG' in df_motion.columns:
                df_motion['accGx'] = df_motion['accG'].apply(lambda x: x['x'])
                df_motion['accGy'] = df_motion['accG'].apply(lambda x: x['y'])
                df_motion['accGz'] = df_motion['accG'].apply(lambda x: x['z'])
                df_motion.drop(columns=['accG'], inplace=True)
                
            # Séparer la colonne 'rotRate' en 'alpha', 'beta', 'gamma'
            if 'rotRate' in df_motion.columns:
                df_motion['alpha'] = df_motion['rotRate'].apply(lambda x: x['alpha'])
                df_motion['beta'] = df_motion['rotRate'].apply(lambda x: x['beta'])
                df_motion['gamma'] = df_motion['rotRate'].apply(lambda x: x['gamma'])
                df_motion.drop(columns=['rotRate'], inplace=True)
                
            # Transformation des colonnes ms pour avoir des timestamp
            def addTS(x):
                return x + start

            if 'msFromStart' in df_motion.columns:
                df_motion['msFromStart'] = df_motion['msFromStart'].apply(addTS)
            if 'msFromStart' in df_orientation.columns:
                df_orientation['msFromStart'] = df_orientation['msFromStart'].apply(addTS)
            if 'msFromStart' in df_heartRate.columns:
                df_heartRate['msFromStart'] = df_heartRate['msFromStart'].apply(addTS)
            if 'msFromStart' in df_cadence.columns:
                df_cadence['msFromStart'] = df_cadence['msFromStart'].apply(addTS)
            
            # Modifications noms de colonnes
            if 'msFromStart' in df_motion.columns :
                df_motion = df_motion.rename(columns={'msFromStart': 'timestamp'})
                df_motion['timestamp'] = df_motion['timestamp']/1000
            
            if 'msFromStart' in df_orientation.columns :
                df_orientation = df_orientation.rename(columns={'msFromStart': 'timestamp'})
                df_orientation['timestamp'] = df_orientation['timestamp']/1000
            
            if 'msFromStart' in df_cadence.columns :
                df_cadence = df_cadence.rename(columns={'msFromStart': 'timestamp'})
                df_cadence['timestamp'] = df_cadence['timestamp']/1000
                df_cadence = df_cadence[['timestamp', 'isRunning', 'instantaneousSpeed', 'instantaneousCadence', 'instantaneousStrideLength', 'totalDistance']]
                
            if 'msFromStart' in df_heartRate.columns :
                df_heartRate = df_heartRate.rename(columns={'msFromStart': 'timestamp'})
                df_heartRate['timestamp'] = df_heartRate['timestamp']/1000
            
            # Écrire chaque DataFrame dans un fichier csv séparé
            directory = 'Results/' + p + '/' + str(i) + '/'

            df_motion.to_csv(directory + p + '_hand_motion_' + str(i) + '.csv', index=False)
            df_orientation.to_csv(directory + p + '_hand_orientation_' + str(i) + '.csv', index=False)
            df_heartRate.to_csv(directory + p + '_hand_heartRate_' + str(i) + '.csv', index=False)
            df_cadence.to_csv(directory + p + '_hand_cadence_' + str(i) + '.csv', index=False)